In [ ]:
# 回到 FINRL/
%cd ..

In [ ]:
from AI_Trading.src import config
from AI_Trading.src.env_portfolio_allocation import *
from AI_Trading.src.evaluate import *
from AI_Trading.src import model_config
from AI_Trading.src.preprocess import *
from AI_Trading.src.testPortfolio import *
from AI_Trading.src.train import *
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3
import pandas as pd
from warnings import simplefilter
simplefilter(action='ignore')
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline

In [ ]:
# data preprocess
train,trade = preprocess(config.TRAIN_START_DATE[0], config.TRAIN_END_DATE[12], config.TEST_START_DATE[12], config.TEST_END_DATE[12])


In [ ]:
env_kwargs = {
        "hmax": 100, 
        "initial_amount": config.INITIAL_AMOUNT, 
        "transaction_cost_pct": 0.001, 
        "state_space": len(train.tic.unique()), 
        "stock_dim": len(train.tic.unique()), 
        "tech_indicator_list": config.INDICATORS, 
        "action_space": len(train.tic.unique()), 
        "reward_scaling": 1e-4,
    }

env_train = portfolioAllocationEnv(df = train, **env_kwargs)


env_trade = portfolioAllocationEnv(df = trade, **env_kwargs)

In [ ]:
# train
model_index = 0
model_name = 'A2C'
trainPortfolioAllocation(env_train, model_name, model_index)

In [ ]:
# load model
model_zip_path = config.TRAINED_MODEL_PATH + model_name + '_' + str(model_index) + '.zip'
if model_name == 'A2C':
    model = A2C.load(model_zip_path)
elif model_name == 'PPO':
    model = PPO.load(model_zip_path)
elif model_name == 'DDPG':
    model = DDPG.load(model_zip_path)
elif model_name == 'TD3':
    model = TD3.load(model_zip_path)
elif model_name == 'SAC':
    model = SAC.load(model_zip_path)

In [ ]:
# test
df_daily_return, df_actions = test_portfolioAllocation(model, env_trade)
df_daily_return.to_csv(config.RESULTS_DIR + 'df_daily_return')
df_actions.to_csv(config.RESULTS_DIR + 'df_actions')

In [ ]:
# evaluate
print("==============DRL Strategy Stats===========")
DRL_returns, DRL_stats = getStats(df_daily_return)
DRL_stats

In [ ]:
print("==============Equal Weight Strategy Stats===========")
equalWeight_actions = getEqualWeightActions(trade)
df_equalWeight_return, df_portfolio_value = computeReturns(equalWeight_actions,trade)
equalWeight_returns, equalWeight_stats = getStats(df_equalWeight_return)
equalWeight_stats

In [ ]:
print("==============Min-Variance Strategy Stats===========")
minVariance, minVariance_stats = getMinVariance(trade)
minVariance_stats

In [ ]:
backtestPlot(DRL_returns, equalWeight_returns)

In [ ]:
print("==============All In Stock Strategy Stats===========")
all_stock_actions= getTicActions(trade, 'VTI')
df_stock_return, df_stock_value = computeReturns(all_stock_actions,trade)
stock_returns, stock_stats = getStats(df_stock_return)
stock_stats

In [ ]:
print("==============All In Debt Strategy Stats===========")
all_debt_actions= getTicActions(trade, 'TLT')
df_debt_return, df_debt_value = computeReturns(all_debt_actions,trade)
debt_returns, debt_stats = getStats(df_debt_return)
debt_stats

In [ ]:
# print("==============All In Debt Strategy Stats===========")
# all_debt, debt_stats = getSingleStats(trade, 'TLT')

In [ ]:
print("==============All In Reit Strategy Stats===========")
all_reit_actions= getTicActions(trade, 'VNQ')
df_reit_return, df_reit_value = computeReturns(all_reit_actions,trade)
reit_returns, reit_stats = getStats(df_reit_return)
reit_stats

In [ ]:
cumulativeReturnPlot(df_daily_return, minVariance, equalWeight_returns, stock_returns, debt_returns, reit_returns)

In [ ]:
weightTrend_plot(df_actions)

In [ ]:

df_VTI = load_data('./AI_Trading/data/VTI.csv', 'VTI')
df_VNQ = load_data('./AI_Trading/data/VNQ.csv', 'VNQ')
df_TLT = load_data('./AI_Trading/data/TLT.csv', 'TLT')

In [ ]:
cumulativeReturnPlot_ETF(stock_returns, debt_returns, reit_returns)

In [ ]:
closePlot_ETF(df_VTI, df_TLT, df_VNQ)